<h1>
    Build Architecture of Deep LSTM
</h1>


<div class="alert alert-block alert-info">
<h3><b>Noted!</b></h3> 
<p>- this model use default batch size : 32 point </p>
</div>

| No. | Hyperparameters | Accuracy |
| :- | -: | :-: |
| 1* | Unit input/Sliding Window | [7,8,9,10] |
| 2* | Unit Neuron/Block LSTM | [1,2,3,4] |
| 3 | Unit Output/Target | 1 |
| 4 | Epochs | 100 |
| 5 | Hidden Layer (Stacked) | 3 |
| 6* | Optimizer | SGD, Adam |
| 7 | Beta Value | B1 = 0,9 & B2 = 0,99 |
| 8* | Learning Rate | 0,1; 0,01; 0,001; 0,0001 |
| 9 | Batch Size | 32 |
| 10* | Dropout | 0%, 20% |
| 11 | Regularize L1 & L2 | 0,01 |
| 12 | Activate Func | sigmoid & tanh |

# Load Libraries and Modules

In [1]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam, SGD


Using TensorFlow backend.


In [2]:
import sys
sys.path.insert(1, '../')

from preprocessing import Preprocessing
from model_adam import Model
from evaluation import Result
import os.path
print("import Preprocessing successfuly !")# -*- coding: utf-8 -*-

import Preprocessing successfuly !


In [3]:
class Slider:
    def __init__(self, filename):
        self.data = {}
        self.slide = [7,8,9,10]
        self.filename = filename

    def dataGenerator(self, slide):
        dataset = '../datasets/'+self.filename
        prep = Preprocessing()
        prep.create_dataset(filename=dataset)
        prep.SplitData()
        prep.MinMaxNorm()
        prep.RollingWindow(window=slide)

        return prep

    def getSlide(self):
        for i in self.slide:
            self.data['slide{}'.format(i)] = self.dataGenerator(i)

        return self.data
    
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

# Load datasets 

In [4]:
usd_slider = Slider(filename='USD_IDR.csv')
yjp_slider = Slider(filename='YJP_IDR.csv')

usd = usd_slider.getSlide()
yjp = yjp_slider.getSlide()

usd_obj = Struct(**usd)
yjp_obj = Struct(**usd)
usd.keys(), yjp.keys()

scaling process was successfully!
scaling process was successfully!
scaling process was successfully!
scaling process was successfully!
scaling process was successfully!
scaling process was successfully!
scaling process was successfully!
scaling process was successfully!


(dict_keys(['slide7', 'slide8', 'slide9', 'slide10']),
 dict_keys(['slide7', 'slide8', 'slide9', 'slide10']))

In [8]:
usd_obj.slide10.X_train

array([[[-0.91541153, -1.        , -0.9926974 , ..., -0.93184239,
         -0.92271413, -0.90750038]],

       [[-1.        , -0.9926974 , -0.97261524, ..., -0.92271413,
         -0.90750038, -0.92453978]],

       [[-0.9926974 , -0.97261524, -0.94827324, ..., -0.90750038,
         -0.92453978, -0.91358588]],

       ...,

       [[ 0.97094173,  0.9775445 ,  0.93488514, ...,  0.94571733,
          0.96683402,  0.95648867]],

       [[ 0.9775445 ,  0.93488514,  0.95162026, ...,  0.96683402,
          0.95648867,  0.95648867]],

       [[ 0.93488514,  0.95162026,  1.        , ...,  0.95648867,
          0.95648867,  0.96598205]]])

# Modeling LSTM/DLSTM
***



## Use 1 Hidden Layer 
***
<div class="alert alert-block alert-success">
<h3><b>Noted!</b></h3> 
<p>- Tunning model base on neuron [1,2,3,4] and sliding window [7,8,9,10]</p>
&#9654 evaluation-usd.ipynb <br>
&#9654 evaluation-yjp.ipynb
</div>

### Init model USD/IDR (One hidden layer)

#### Set params

In [8]:
# usd environment
slides  = [7,8,9,10]
neurons = [1,2,3,4]
modelpaths = './newmodels/usd/'
models = {}

In [23]:
for slide in slides:
    lookback = usd['slide{}'.format(slide)].X_train.shape[2]
    for neuron in neurons:
        model=Sequential()
        model.add(LSTM(neuron, input_shape=(1, lookback)))
        model.add(Dense(1))
        models['{},{}'.format(slide, neuron)]=model

#### Save models

In [24]:
modelpath='./newmodels/usd'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model7,1 successfully!
save figure model7,2 successfully!
save figure model7,3 successfully!
save figure model7,4 successfully!
save figure model8,1 successfully!
save figure model8,2 successfully!
save figure model8,3 successfully!
save figure model8,4 successfully!
save figure model9,1 successfully!
save figure model9,2 successfully!
save figure model9,3 successfully!
save figure model9,4 successfully!
save figure model10,1 successfully!
save figure model10,2 successfully!
save figure model10,3 successfully!
save figure model10,4 successfully!


#### Fitting models

In [25]:
# models fit and compile
neurons = [1,2,3,4]
slides  = [7,8,9,10]
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'usd', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=usd['slide{}'.format(key[:-2])].X_train
        y_train=usd['slide{}'.format(key[:-2])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)



=====] - 0s 115us/step - loss: 0.0021
Epoch 72/100
1438/1438 [==============================] - 0s 121us/step - loss: 0.0021
Epoch 73/100
1438/1438 [==============================] - 0s 125us/step - loss: 0.0020
Epoch 74/100
1438/1438 [==============================] - 0s 110us/step - loss: 0.0020
Epoch 75/100
1438/1438 [==============================] - 0s 109us/step - loss: 0.0020
Epoch 76/100
1438/1438 [==============================] - 0s 110us/step - loss: 0.0020
Epoch 77/100
1438/1438 [==============================] - 0s 109us/step - loss: 0.0019
Epoch 78/100
1438/1438 [==============================] - 0s 115us/step - loss: 0.0019
Epoch 79/100
1438/1438 [==============================] - 0s 109us/step - loss: 0.0019
Epoch 80/100
1438/1438 [==============================] - 0s 116us/step - loss: 0.0019
Epoch 81/100
1438/1438 [==============================] - 0s 108us/step - loss: 0.0019
Epoch 82/100
1438/1438 [==============================] - 0s 112us/step - loss: 0.0019
Epoch

### Init model YJP/IDR (One hidden layer)

#### Set params

In [ ]:
# yjp environment

slides  = [7,8,9,10]
neurons = [1,2,3,4]
modelpaths = './newmodels/yjp/'
models = {}

In [27]:
for slide in slides:
    lookback = yjp['slide{}'.format(slide)].X_train.shape[2]
    for neuron in neurons:
        model=Sequential()
        model.add(LSTM(neuron, input_shape=(1, lookback)))
        model.add(Dense(1))
        models['{},{}'.format(slide, neuron)]=model

#### Save models

In [28]:
modelpath='./newmodels/yjp'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model7,1 successfully!
save figure model7,2 successfully!
save figure model7,3 successfully!
save figure model7,4 successfully!
save figure model8,1 successfully!
save figure model8,2 successfully!
save figure model8,3 successfully!
save figure model8,4 successfully!
save figure model9,1 successfully!
save figure model9,2 successfully!
save figure model9,3 successfully!
save figure model9,4 successfully!
save figure model10,1 successfully!
save figure model10,2 successfully!
save figure model10,3 successfully!
save figure model10,4 successfully!


#### Fitting models

In [29]:
# models fit and compile
neurons = [1,2,3,4]
slides  = [7,8,9,10]
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'yjp', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=yjp['slide{}'.format(key[:-2])].X_train
        y_train=yjp['slide{}'.format(key[:-2])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)



=========] - 0s 140us/step - loss: 0.0037
Epoch 72/100
1439/1439 [==============================] - 0s 137us/step - loss: 0.0037
Epoch 73/100
1439/1439 [==============================] - 0s 136us/step - loss: 0.0036
Epoch 74/100
1439/1439 [==============================] - 0s 156us/step - loss: 0.0036
Epoch 75/100
1439/1439 [==============================] - 0s 139us/step - loss: 0.0035
Epoch 76/100
1439/1439 [==============================] - 0s 140us/step - loss: 0.0035
Epoch 77/100
1439/1439 [==============================] - 0s 145us/step - loss: 0.0035
Epoch 78/100
1439/1439 [==============================] - 0s 137us/step - loss: 0.0035
Epoch 79/100
1439/1439 [==============================] - 0s 146us/step - loss: 0.0034
Epoch 80/100
1439/1439 [==============================] - 0s 142us/step - loss: 0.0034
Epoch 81/100
1439/1439 [==============================] - 0s 138us/step - loss: 0.0033
Epoch 82/100
1439/1439 [==============================] - 0s 152us/step - loss: 0.0033
E

## Use **2** Hidden Layers
***
<div class="alert alert-block alert-success">
<h3><b>Noted!</b></h3> 
<p>- One layer previosly selected on the predictive evaluation stage </p>
&#9654 evaluation-usd.ipynb <br>
&#9654 evaluation-yjp.ipynb
</div>

### Init Model USD/IDR (2 Hidden Layers)

#### Set Params

In [14]:
hidden2 = 4
bestSlide = 9

#### Save models

In [22]:
lookback = usd['slide{}'.format(bestSlide)].X_train.shape[2]
for neuron in neurons:
    model=Sequential()
    model.add(LSTM(hidden2, input_shape=(1, 9), return_sequences=True))
    model.add(LSTM(neuron))
    model.add(Dense(1))
    models['{},{},{}'.format(9,4,neuron)]=model

In [23]:
models

{'9,4,1': <keras.engine.sequential.Sequential at 0x7fdeb0dcab38>,
 '9,4,2': <keras.engine.sequential.Sequential at 0x7fdeb0d13a90>,
 '9,4,3': <keras.engine.sequential.Sequential at 0x7fdeb0bb7be0>,
 '9,4,4': <keras.engine.sequential.Sequential at 0x7fdeb0adebe0>}

In [12]:
modelpath='./newmodels/usd'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model9,4,1 successfully!
save figure model9,4,2 successfully!
save figure model9,4,3 successfully!
save figure model9,4,4 successfully!


#### Fitting models

In [14]:
# models fit and compile
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'usd', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=usd['slide{}'.format(key.split(',')[0])].X_train
        y_train=usd['slide{}'.format(key.split(',')[0])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)



=======] - 0s 232us/step - loss: 0.0015
Epoch 72/100
1439/1439 [==============================] - 0s 190us/step - loss: 0.0015
Epoch 73/100
1439/1439 [==============================] - 0s 188us/step - loss: 0.0015
Epoch 74/100
1439/1439 [==============================] - 0s 223us/step - loss: 0.0015
Epoch 75/100
1439/1439 [==============================] - 0s 206us/step - loss: 0.0015
Epoch 76/100
1439/1439 [==============================] - 0s 186us/step - loss: 0.0015
Epoch 77/100
1439/1439 [==============================] - 0s 191us/step - loss: 0.0015
Epoch 78/100
1439/1439 [==============================] - 0s 198us/step - loss: 0.0015
Epoch 79/100
1439/1439 [==============================] - 0s 201us/step - loss: 0.0015
Epoch 80/100
1439/1439 [==============================] - 0s 194us/step - loss: 0.0014
Epoch 81/100
1439/1439 [==============================] - 0s 203us/step - loss: 0.0014
Epoch 82/100
1439/1439 [==============================] - 0s 200us/step - loss: 0.0014
Epo

### Init model YJP/IDR (2  Hidden Layers)

In [28]:
neurons = [1,2,3,4]
models = {}

#### Set Params

In [29]:
bestSlide = 8
hidden1 = 4

#### Save models

In [30]:
lookback = yjp['slide{}'.format(bestSlide)].X_train.shape[2]
for neuron in neurons:
    model=Sequential()
    model.add(LSTM(hidden2, input_shape=(1, bestSlide), return_sequences=True))
    model.add(LSTM(neuron))
    model.add(Dense(1))
    models['{},{},{}'.format(bestSlide,4,neuron)]=model

models

{'8,4,1': <keras.engine.sequential.Sequential at 0x7fdeb0487e10>,
 '8,4,2': <keras.engine.sequential.Sequential at 0x7fdeb03d19b0>,
 '8,4,3': <keras.engine.sequential.Sequential at 0x7fdeb02f39b0>,
 '8,4,4': <keras.engine.sequential.Sequential at 0x7fdeb019ab38>}

In [31]:
modelpath='./newmodels/yjp/'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model8,4,1 successfully!
save figure model8,4,2 successfully!
save figure model8,4,3 successfully!
save figure model8,4,4 successfully!


#### Fitting models

In [32]:
# models fit and compile
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'yjp', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=yjp['slide{}'.format(key.split(',')[0])].X_train
        y_train=yjp['slide{}'.format(key.split(',')[0])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)



newmodels/yjp/model8,4,1.h5

Epoch 1/100
1441/1441 [==============================] - 1s 941us/step - loss: 0.2099
Epoch 2/100
1441/1441 [==============================] - 0s 111us/step - loss: 0.1876
Epoch 3/100
1441/1441 [==============================] - 0s 121us/step - loss: 0.1623
Epoch 4/100
1441/1441 [==============================] - 0s 115us/step - loss: 0.1289
Epoch 5/100
1441/1441 [==============================] - 0s 118us/step - loss: 0.0933
Epoch 6/100
1441/1441 [==============================] - 0s 124us/step - loss: 0.0626
Epoch 7/100
1441/1441 [==============================] - 0s 124us/step - loss: 0.0399
Epoch 8/100
1441/1441 [==============================] - 0s 129us/step - loss: 0.0267
Epoch 9/100
1441/1441 [==============================] - 0s 124us/step - loss: 0.0201
Epoch 10/100
1441/1441 [==============================] - 0s 126us/step - loss: 0.0170
Epoch 11/100
1441/1441 [==============================] - 0s 123us/step - loss: 0.0153
Epoch 12/100
1441/1441 

## Use 3 Hidden Layers
***
<div class="alert alert-block alert-success">
<h3><b>Noted!</b></h3> 
<p>- One layer previosly selected on the predictive evaluation stage </p>
&#9654 evaluation-usd.ipynb <br>
&#9654 evaluation-yjp.ipynb
</div>

### Init model USD/IDR (3 Hidden layers)

#### Set params

In [13]:
neurons = [1,2,3,4]
slides  = [7,8,9,10]
models  = {}

In [12]:
hidden1 = 4
hidden2 = 3
bestSlide = 9

#### Save models

In [14]:
lookback = usd['slide{}'.format(bestSlide)].X_train.shape[2]
for neuron in neurons:
    model=Sequential()
    model.add(LSTM(hidden1, input_shape=(1, 9), return_sequences=True))
    model.add(LSTM(hidden2, return_sequences=True))
    model.add(LSTM(neuron))
    model.add(Dense(1))
    models['{},{},{},{}'.format(9,4,3,neuron)]=model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
models

{'9,4,3,1': <keras.engine.sequential.Sequential at 0x7f01a0e73048>,
 '9,4,3,2': <keras.engine.sequential.Sequential at 0x7f01a04baac8>,
 '9,4,3,3': <keras.engine.sequential.Sequential at 0x7f01a0371630>,
 '9,4,3,4': <keras.engine.sequential.Sequential at 0x7f01a01a5a20>}

In [17]:
modelpath='./newmodels/usd'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model9,4,3,1 successfully!
save figure model9,4,3,2 successfully!
save figure model9,4,3,3 successfully!
save figure model9,4,3,4 successfully!


#### Fitting models

In [18]:
# models fit and compile
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'usd', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=usd['slide{}'.format(key.split(',')[0])].X_train
        y_train=usd['slide{}'.format(key.split(',')[0])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)

===] - 0s 238us/step - loss: 0.0015
Epoch 72/100
1439/1439 [==============================] - 0s 230us/step - loss: 0.0015
Epoch 73/100
1439/1439 [==============================] - 0s 214us/step - loss: 0.0015
Epoch 74/100
1439/1439 [==============================] - 0s 266us/step - loss: 0.0014
Epoch 75/100
1439/1439 [==============================] - 0s 237us/step - loss: 0.0015
Epoch 76/100
1439/1439 [==============================] - 0s 227us/step - loss: 0.0014
Epoch 77/100
1439/1439 [==============================] - 0s 226us/step - loss: 0.0014
Epoch 78/100
1439/1439 [==============================] - 0s 216us/step - loss: 0.0014
Epoch 79/100
1439/1439 [==============================] - 0s 219us/step - loss: 0.0014
Epoch 80/100
1439/1439 [==============================] - 0s 222us/step - loss: 0.0014
Epoch 81/100
1439/1439 [==============================] - 0s 221us/step - loss: 0.0014
Epoch 82/100
1439/1439 [==============================] - 0s 228us/step - loss: 0.0014
Epoch 8

### Init model YJP/IDR (3 Hidden layers)

#### Set params

In [34]:
neurons = [1,2,3,4]
slides  = [7,8,9,10]
models  = {}

In [33]:
hidden1 = 4
hidden2 = 4
bestSlide = 8

#### Save models

In [35]:
lookback = yjp['slide{}'.format(bestSlide)].X_train.shape[2]
for neuron in neurons:
    model=Sequential()
    model.add(LSTM(hidden1, input_shape=(1, bestSlide), return_sequences=True))
    model.add(LSTM(hidden2, return_sequences=True))
    model.add(LSTM(neuron))
    model.add(Dense(1))
    models['{},{},{},{}'.format(bestSlide, hidden1, hidden2, neuron)]=model

In [36]:
models

{'8,4,4,1': <keras.engine.sequential.Sequential at 0x7fde905ab208>,
 '8,4,4,2': <keras.engine.sequential.Sequential at 0x7fde903c7fd0>,
 '8,4,4,3': <keras.engine.sequential.Sequential at 0x7fde901f9780>,
 '8,4,4,4': <keras.engine.sequential.Sequential at 0x7fde900a7ac8>}

In [37]:
modelpath='./newmodels/yjp'
for key, value in models.items():
    plot_model(model=value, show_shapes=True, to_file=modelpath+'/imageModels/model{}.png'.format(key))
    print("save figure model{} successfully!".format(key))

save figure model8,4,4,1 successfully!
save figure model8,4,4,2 successfully!
save figure model8,4,4,3 successfully!
save figure model8,4,4,4 successfully!


#### Fitting models

In [38]:
# models fit and compile
for key, value in models.items():
    modelpath = 'newmodels/{}/model{}.h5'.format(
        'yjp', key
    )
    if os.path.exists(modelpath):
        print("model {} is exist!".format(modelpath))
    else:
        print(modelpath)
        X_train=yjp['slide{}'.format(key.split(',')[0])].X_train
        y_train=yjp['slide{}'.format(key.split(',')[0])].y_train
        value.compile(loss='mse',optimizer='adam')
        value.fit(X_train, y_train, epochs=100, batch_size=32)
        value.save(modelpath)

newmodels/yjp/model8,4,4,1.h5
Epoch 1/100
1441/1441 [==============================] - 2s 1ms/step - loss: 0.2074
Epoch 2/100
1441/1441 [==============================] - 0s 162us/step - loss: 0.2060
Epoch 3/100
1441/1441 [==============================] - 0s 174us/step - loss: 0.2026
Epoch 4/100
1441/1441 [==============================] - 0s 164us/step - loss: 0.1939
Epoch 5/100
1441/1441 [==============================] - 0s 163us/step - loss: 0.1752
Epoch 6/100
1441/1441 [==============================] - 0s 172us/step - loss: 0.1458
Epoch 7/100
1441/1441 [==============================] - 0s 163us/step - loss: 0.1120
Epoch 8/100
1441/1441 [==============================] - 0s 180us/step - loss: 0.0830
Epoch 9/100
1441/1441 [==============================] - 0s 170us/step - loss: 0.0624
Epoch 10/100
1441/1441 [==============================] - 0s 168us/step - loss: 0.0488
Epoch 11/100
1441/1441 [==============================] - 0s 167us/step - loss: 0.0394
Epoch 12/100
1441/1441 [